In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
import os
import csv
from dateutil.parser import parse

In [2]:
df_train = pd.read_csv("../../data/kalapa/train.csv")
df_test = pd.read_csv("../../data/kalapa/test.csv")

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def check_duplicate(feature1, feature2):
    check1 = list(df_train[feature1].fillna("NaN"))
    check2 = list(df_train[feature2].fillna("NaN"))
    return check1 == check2
distinct_columns = {}
for col1 in df_train.columns:
    vc = df_train[col1].value_counts()
    index = [len(vc), set(list(vc)), set(list(vc.index))]
    for col2 in distinct_columns:
        if index == distinct_columns[col2] and check_duplicate(col1, col2):
            break
    else:
        distinct_columns[col1] = index
df_train = df_train[distinct_columns]
del distinct_columns['label']
df_test = df_test[distinct_columns]

In [4]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G"],
    "cat_str" : ["Field_4", "Field_12", "gioiTinh", "Field_62", "Field_65", "Field_66",\
                 "info_social_sex", "Field_36", "Field_54", "Field_55", "Field_38", "brief", "Field_61", \
                 'data.basic_info.locale', "currentLocationCountry", 'homeTownCountry', "currentLocationState", \
                'homeTownState'], 
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40", \
             "Field_43", "Field_44", "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate",  ],
    "text" : ["diaChi", "Field_46", "Field_48", "Field_49", "Field_56","Field_18",\
              "maCv", "currentLocationName",'homeTownName', "currentLocationCity", "homeTownCity"],
    "drop" : [ "Field_34","Field_68", "Field_45"]
}

In [5]:
def fix34(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")

In [6]:
classes_47 = {
    "Zezo" : 0,
    "One" : 1,
    "Two" : 2,
    "Three": 3,
}

In [7]:
classes_62 = {
    "I" : 1,
    "II" : 2,
    "III": 3,
    "IV" : 4,
    "V" : 5
}

In [8]:
for data_set in [df_train, df_test]:
    data_set["Field_34"] = data_set["Field_34"].map(lambda x: fix34(x))
    data_set["Field_47"] = data_set["Field_47"].map(lambda x: classes_47[x] if x in classes_47 else np.nan)
    data_set["Field_62"] = data_set["Field_62"].map(lambda x: classes_62[x] if x in classes_62 else np.nan)
    data_set["ngaySinh"] = data_set["ngaySinh"].fillna("NaN")
    data_set["ngaySinh"] = data_set["ngaySinh"].map(lambda x: parse(str(x)[:8]) if x is not "NaN" else np.nan)

In [9]:
df_train["Field_34"] = df_train["Field_34"].dt.strftime('%m-%Y')
df_test["Field_34"] = df_test["Field_34"].dt.strftime('%m-%Y')

In [10]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53030 entries, 0 to 53029
Columns: 149 entries, id to Field_82
dtypes: datetime64[ns](1), float64(88), int64(3), object(57)
memory usage: 60.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20381 entries, 0 to 20380
Columns: 148 entries, id to Field_82
dtypes: datetime64[ns](1), float64(88), int64(2), object(57)
memory usage: 23.0+ MB


In [11]:
data_version = "01-10-2020"

In [12]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
df_train.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
df_test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)